In [1]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from tqdm import tqdm
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 10개 파일을 불러왔습니다: 1201.csv, 137.csv, 281.csv, 397.csv, 401.csv, 430.csv, 514.csv, 524.csv, 775.csv, 89.csv
📊 데이터 Shape: I=10, T=24, S=20
✅ 시뮬레이션 초기화 완료: S=20, Randomness='high', M1=768.00, M2=2463.00


Holistic Optimization

In [3]:
set = gp.Model("set")
set.setParam("MIPGap", 1e-7)

x_hol = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
ep_hol = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
em_hol = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_minus")

yp_hol = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym_hol = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z_hol = set.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
zc_hol = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd_hol = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
d_hol = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")

p1_hol = set.addVars(I, T, S, vtype=GRB.BINARY, name="p1")
p2_hol = set.addVars(I, T, S, vtype=GRB.BINARY, name="p2")
p3_hol = set.addVars(I, T, S, vtype=GRB.BINARY, name="p3")
p4_hol = set.addVars(I, T, S, vtype=GRB.BINARY, name="p4")

set.update()

obj_hol = gp.quicksum(P_DA[t] * gp.quicksum(x_hol[i, t] for i in range(I)) for t in range(T)) + gp.quicksum((1 / S) * (P_RT[t, s] * gp.quicksum(ep_hol[i, t, s] for i in range(I)) - P_PN[t] * gp.quicksum(em_hol[i, t, s] for i in range(I))) for t in range(T) for s in range(S))

set.setObjective(obj_hol, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(R[i, t, s] - x_hol[i, t] == yp_hol[i, t, s] - ym_hol[i, t, s] + zc_hol[i, t, s] - zd_hol[i, t, s])
    set.addConstr(yp_hol[i, t, s] + zc_hol[i, t, s] <= R[i, t, s] + zd_hol[i, t, s])
    set.addConstr(zd_hol[i, t, s] <= z_hol[i, t, s])
    set.addConstr(zc_hol[i, t, s] <= K[i] - z_hol[i, t, s])
    set.addConstr(yp_hol[i, t, s] <= M1 * p3_hol[i, t, s])
    set.addConstr(ym_hol[i, t, s] <= M1 * (1 - p3_hol[i, t, s]))
    set.addConstr(ym_hol[i, t, s] <= M1 * p2_hol[i, t, s])
    set.addConstr(zc_hol[i, t, s] <= M1 * (1 - p2_hol[i, t, s]))
    set.addConstr(zc_hol[i, t, s] <= M1 * p1_hol[i, t, s])
    set.addConstr(zd_hol[i, t, s] <= M1 * (1 - p1_hol[i, t, s]))
    set.addConstr(z_hol[i, t, s] <= K[i])
    set.addConstr(z_hol[i, t + 1, s] == z_hol[i, t, s] + zc_hol[i, t, s] - zd_hol[i, t, s])
for i, s in product(range(I), range(S)):
    set.addConstr(z_hol[i, 0, s] == K0[i])

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(ep_hol[i, t, s] == yp_hol[i, t, s] - gp.quicksum(d_hol[i, j, t, s] for j in range(I)))
    set.addConstr(em_hol[i, t, s] == ym_hol[i, t, s] - gp.quicksum(d_hol[j, i, t, s] for j in range(I)))
    set.addConstr(gp.quicksum(ep_hol[i, t, s] for i in range(I)) <= M2 * p4_hol[i, t, s])
    set.addConstr(gp.quicksum(em_hol[i, t, s] for i in range(I)) <= M2 * (1 - p4_hol[i, t, s]))
    set.addConstr(d_hol[i, i, t, s] == 0)

set.optimize()

if set.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {set.objVal}")
else:
    print("No optimal solution found.")
    
x_hol = np.array([[x_hol[i, t].X for t in range(T)] for i in range(I)])
a_hol = np.sum(x_hol, axis=0)
yp_hol = np.array([[[yp_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
ym_hol = np.array([[[ym_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
z_hol = np.array([[[z_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
zc_hol = np.array([[[zc_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
zd_hol = np.array([[[zd_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
ep_hol = np.array([[[ep_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
bp_hol = np.sum(ep_hol, axis=0) 
em_hol = np.array([[[em_hol[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
bm_hol = np.sum(em_hol, axis=0) 
d_hol = np.array([[[[d_hol[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)] for i in range(I)])
dp_hol = np.sum(d_hol, axis=1)
dm_hol = np.sum(d_hol, axis=0)

Set parameter MIPGap to value 1e-07
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
MIPGap  1e-07

Optimize a model with 81800 rows, 101240 columns and 365000 nonzeros
Model fingerprint: 0xaccb9321
Variable types: 82040 continuous, 19200 integer (19200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 42679 rows and 40264 columns
Presolve time: 0.35s
Presolved: 39121 rows, 60976 columns, 167335 nonzeros
Variable types: 51651 continuous, 9325 integer (9325 binary)
Found heuristic solution: objective 1159977.4189
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved w